In [82]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as sparse
import scipy.special     as sp
import hylife.utilitis_FEEC.bsplines             as bsp
import hylife.utilitis_FEEC.evaluation           as eva
import hylife.geometry.mappings_analytical       as mapping
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import time
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [83]:
def Parameters(Nel       = [8,8, 2],              # mesh generation on logical domain
               bc        = [True, True, True],      # boundary conditions (True: periodic, False: else)
               p         = [3, 4, 1],         # spline degrees 
               Nq        = [1, 6, 2],
               Lx        = 4.,
               Ly        = 2.,
               Lz        = 1.,
               kind_map  = 1,
               alpha     =0.05):# number of quadrature points per element


    el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
    delta     = [1/Nel for Nel in Nel]                                            # element sizes
    T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
    t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
    NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
    NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]              #number of basis functions (D functions)

    if kind_map == 1:               #slab geometry
        args_map = [kind_map, [Lx,Ly,Lz]]
    elif kind_map ==3:
        args_map = kind_map, [Lx,Ly,alpha,Lz]

    print('Parameters update Realized')
    return Nel, bc, p, Nq, el_b, delta, T, t, NbaseN, NbaseD, Lx, Ly, Lz, kind_map, args_map, alpha   

In [84]:
def Projectors_ini():
    global T, p, bc, Nq
    import hylife.utilitis_FEEC.projectors_global    as projectors_global
    import hylife.utilitis_FEEC.derivatives          as derivatives
    proj_glob = projectors_global.projectors_3d(T, p, bc,Nq)
    proj_glob.assemble_V3()
    proj_glob.assemble_V2()
    proj_glob.assemble_V1()
    der = derivatives.discrete_derivatives(T, p, bc)
    DIV = der.DIV_3d()
    CUR = der.CURL_3d()
    print('Projectors Initialized')
    return proj_glob, der, DIV, CUR

In [85]:
(Nel, bc, p, Nq, el_b, 
 delta, T, t, NbaseN, 
 NbaseD, Lx, Ly, Lz, kind_map,
 args_map, alpha)=Parameters(Nel=[8,4,2])
(proj_glob, der, DIV, CUR)=Projectors_ini()

Parameters update Realized
Projectors Initialized


In [86]:
B_phys = lambda x,y,z: [np.sin(2*np.pi*x/Lx),0,0]

In [87]:
def B_pull_back(xi1,xi2,xi3):
    x = [mapping.f(xi1, xi2, xi3, *args_map, j) for j in [1,2,3]]
    det_df=mapping.det_df(xi1, xi2, xi3, *args_map)
    DF_inv=np.array([[mapping.df_inv(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
             for j in [1,2,3]])
    B_2=det_df*DF_inv.dot(B_phys(x[0],x[1],x[2]))
    return B_2

In [88]:
def B_x(xi1, xi2_arr, xi3_arr):
    n2 = xi2_arr.shape[0]
    n3 = xi3_arr.shape[0]
    Bx_2=np.transpose(np.array([[B_pull_back(xi1,xi2_arr[i2],xi3_arr[i3])[0]for i2 in range(n2)]
                   for i3 in range(n3)]))
    return Bx_2
def B_y(xi1_arr, xi2, xi3_arr):
    n1 = xi1_arr.shape[0]
    n3 = xi3_arr.shape[0]
    By_2=np.transpose(np.array([[B_pull_back(xi1_arr[i1],xi2,xi3_arr[i3])[1]for i1 in range(n1)]
                   for i3 in range(n3)]))
    return By_2
def B_z(xi1_arr, xi2_arr, xi3):
    n1 = xi1_arr.shape[0]
    n2 = xi2_arr.shape[0]
    Bz_2=np.transpose(np.array([[B_pull_back(xi1_arr[i1],xi2_arr[i2],xi3)[2] for i1 in range(n1)]
                   for i2 in range(n2)]))
    return Bz_2

In [89]:
def B_Bh_proj():
    ##Project B as a 2 form
    Bx_coeff=proj_glob.PI_21(B_x)
    By_coeff=proj_glob.PI_22(B_y)
    Bz_coeff=proj_glob.PI_23(B_z)
    return Bx_coeff, By_coeff, Bz_coeff

#### $DF$, $DF^T$, $DF^{-1}$,$G$, $det(DF)$ in logical coordinates

In [90]:
xi1, xi2, xi3=el_b[0], el_b[1], el_b[2]
DF=np.array([[mapping.df(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
                 for j in [1,2,3]])
DF_inv=np.array([[mapping.df_inv(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
             for j in [1,2,3]])
det_df=mapping.det_df(xi1, xi2, xi3, *args_map)
DF_T=np.transpose(DF)
G=DF_T.dot(DF)
G_inv=DF_inv.dot(np.transpose(DF_T))
def hodge_op(coeff): #2-form
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    coeff_r=coeff.reshape(dim,3)
    B_l = np.array([-1/(det_df)*G.dot(coeff_r[k]) 
                     for k in range(dim)]).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return B_l
def hash_op(coeff):
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    coeff_r=coeff.reshape(dim,3)
    B_v = np.array([G_inv.dot(coeff_r[k]) 
                    for k in range(dim)]).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return B_v

#### Projections Initialization

In [91]:
Bx_coeff, By_coeff, Bz_coeff= B_Bh_proj()
B_2form=np.array(list(zip(Bx_coeff.flatten(),By_coeff.flatten(),Bz_coeff.flatten())))

In [92]:
dt=0.01
dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
for i in range(3):
    B_1form=hodge_op(B_2form)
    curlB=CUR.dot(np.concatenate((B_1form.reshape(dim,3)[:,0],
                    B_1form.reshape(dim,3)[:,1],B_1form.reshape(dim,3)[:,2])))
    J=hodge_op(curlB)
    
    J1=hash_op(J)
    i_JB=np.cross(J1.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    v=hash_op(i_JB)
    vxB=np.cross(v.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    cur_vxB=CUR.dot(np.concatenate((vxB.reshape(dim,3)[:,0],
                            vxB.reshape(dim,3)[:,1],vxB.reshape(dim,3)[:,2]))) ##Different Num elements
    B_t=cur_vxB.reshape(dim,3)
    B_2form=B_2form+dt*B_t
    print(B_2form[0])

[-0.19590291  0.          0.        ]
[-1.95902906e-01  1.15613662e-24  2.86869862e-24]
[-1.95902906e-01  3.46948441e-24  8.60358829e-24]


In [ ]:
cd hylife

In [5]:
import Test_Operators_MF as TO

Parameters update Realized
Projectors Initialized


In [51]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as sparse
import scipy.special     as sp
import hylife.utilitis_FEEC.bsplines             as bsp
import hylife.utilitis_FEEC.evaluation           as eva
import hylife.geometry.mappings_analytical       as mapping

def Parameters(Nel       = [8,8, 2],              # mesh generation on logical domain
               bc        = [True, True, True],      # boundary conditions (True: periodic, False: else)
               p         = [3, 4, 1],         # spline degrees 
               Nq        = [1, 6, 2],
               Lx        = 4.,
               Ly        = 2.,
               Lz        = 1.,
               kind_map  = 1,
               alpha     =0.05):# number of quadrature points per element


    el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
    delta     = [1/Nel for Nel in Nel]                                            # element sizes
    T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
    t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
    NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
    NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]              #number of basis functions (D functions)

    if kind_map == 1:               #slab geometry
        args_map = [kind_map, [Lx,Ly,Lz]]
    elif kind_map ==3:
        args_map = kind_map, [Lx,Ly,alpha,Lz]

    print('Parameters update Realized')
    return Nel, bc, p, Nq, el_b, delta, T, t, NbaseN, NbaseD, Lx, Ly, Lz, kind_map, args_map, alpha   

def Projectors_ini():
    global T, p, bc, Nq
    import hylife.utilitis_FEEC.projectors_global    as projectors_global
    import hylife.utilitis_FEEC.derivatives          as derivatives
    proj_glob = projectors_global.projectors_3d(T, p, bc,Nq)
    proj_glob.assemble_V3()
    proj_glob.assemble_V2()
    proj_glob.assemble_V1()
    der = derivatives.discrete_derivatives(T, p, bc)
    DIV = der.DIV_3d()
    CUR = der.CURL_3d()
    print('Projectors Initialized')
    return proj_glob, der, DIV, CUR

##Initialization 
(Nel, bc, p, Nq, el_b, 
     delta, T, t, NbaseN, 
     NbaseD, Lx, Ly, Lz, kind_map,
     args_map, alpha)=Parameters(Nel=[4,4,2])
(proj_glob, der, DIV, CUR)=Projectors_ini()

xi1, xi2, xi3=el_b[0], el_b[1], el_b[2]
DF=np.array([[mapping.df(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
                 for j in [1,2,3]])
DF_inv=np.array([[mapping.df_inv(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
             for j in [1,2,3]])
det_df=mapping.det_df(xi1, xi2, xi3, *args_map)
DF_T=np.transpose(DF)
print(DF.shape)
G=DF_T.dot(DF)
G_inv=DF_inv.dot(np.transpose(DF_T))

B_phys = lambda x,y,z: [np.sin(2*np.pi*x/Lx),np.cos(2*np.pi*x/Ly),2*np.sin(2*np.pi*x/Lz)]

### Functions for testins
def B_pull_back(xi1,xi2,xi3):
    global B_phys
    x = [mapping.f(xi1, xi2, xi3, *args_map, j) for j in [1,2,3]]
    det_df=mapping.det_df(xi1, xi2, xi3, *args_map)
    DF_inv=np.array([[mapping.df_inv(xi1, xi2, xi3, *args_map, (i)*10+(j)) for i in [1,2,3]]
             for j in [1,2,3]])
    B_2=det_df*DF_inv.dot(B_phys(x[0],x[1],x[2]))
    return B_2
def B_x(xi1, xi2_arr, xi3_arr):
    n2 = xi2_arr.shape[0]
    n3 = xi3_arr.shape[0]
    Bx_2=np.transpose(np.array([[B_pull_back(xi1,xi2_arr[i2],xi3_arr[i3])[0]for i2 in range(n2)]
                   for i3 in range(n3)]))
    return Bx_2
def B_y(xi1_arr, xi2, xi3_arr):
    n1 = xi1_arr.shape[0]
    n3 = xi3_arr.shape[0]
    By_2=np.transpose(np.array([[B_pull_back(xi1_arr[i1],xi2,xi3_arr[i3])[1]for i1 in range(n1)]
                   for i3 in range(n3)]))
    return By_2
def B_z(xi1_arr, xi2_arr, xi3):
    n1 = xi1_arr.shape[0]
    n2 = xi2_arr.shape[0]
    Bz_2=np.transpose(np.array([[B_pull_back(xi1_arr[i1],xi2_arr[i2],xi3)[2] for i1 in range(n1)]
                   for i2 in range(n2)]))
    return Bz_2
def B_Bh_proj():
    ##Project B as a 2 form
    Bx_coeff=proj_glob.PI_21(B_x)
    By_coeff=proj_glob.PI_22(B_y)
    Bz_coeff=proj_glob.PI_23(B_z)
    return Bx_coeff, By_coeff, Bz_coeff

def hodge_op(coeff): #2-form
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    coeff_r=coeff.reshape(dim,3)
    B_l = np.array([-1/(det_df)*G.dot(coeff_r[k]) 
                     for k in range(dim)]).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return B_l
def hash_op(coeff):
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    coeff_r=coeff.reshape(dim,3)
    B_v = np.array([G_inv.dot(coeff_r[k]) 
                    for k in range(dim)]).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return B_v
def curlB_f(B_ini):
    global B_phys
    B_phys=B_ini
    xi1, xi2, xi3=el_b[0], el_b[1], el_b[2]
    Bx_coeff, By_coeff, Bz_coeff= B_Bh_proj()
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    B_2form=np.array(list(zip(Bx_coeff.flatten(),By_coeff.flatten(),Bz_coeff.flatten())))
    B_1form=hodge_op(B_2form)
    curlB=CUR.dot(np.concatenate((B_1form.reshape(dim,3)[:,0],
                   B_1form.reshape(dim,3)[:,1],B_1form.reshape(dim,3)[:,2]))).reshape(dim,3)
    curlB_phys=1/det_df*np.array([DF.dot(curlB[k]) 
                    for k in range(dim)]).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return curlB_phys
def JxB (B_ini):
    xi1, xi2, xi3=el_b[0], el_b[1], el_b[2]
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    Bx_coeff, By_coeff, Bz_coeff= B_Bh_proj()
    B_2form=np.array(list(zip(Bx_coeff.flatten(),By_coeff.flatten(),Bz_coeff.flatten())))
    J=hodge_op(curlB(B_ini))
    J1=hash_op(J)
    i_JB=np.cross(J1.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return i_JB
def vxB_f(B_ini):
    xi1, xi2, xi3=el_b[0], el_b[1], el_b[2]
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    Bx_coeff, By_coeff, Bz_coeff= B_Bh_proj()
    B_2form=np.array(list(zip(Bx_coeff.flatten(),By_coeff.flatten(),Bz_coeff.flatten())))
    J=hodge_op(curlB(B_ini))
    J1=hash_op(J)
    i_JB=np.cross(J1.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    v=hash_op(i_JB)
    vxB=np.cross(v.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    return vxB
def curl_vxB(B_ini):
    xi1, xi2, xi3=el_b[0], el_b[1], el_b[2]
    dim=NbaseN[0]*NbaseN[1]*NbaseN[2]
    Bx_coeff, By_coeff, Bz_coeff= B_Bh_proj()
    B_2form=np.array(list(zip(Bx_coeff.flatten(),By_coeff.flatten(),Bz_coeff.flatten())))
    J=hodge_op(curlB(B_ini))
    J1=hash_op(J)
    i_JB=np.cross(J1.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    v=hash_op(i_JB)
    vxB=np.cross(v.reshape(dim,3),B_2form).reshape(NbaseN[0],NbaseN[1],NbaseN[2],3)
    cur_vxB=CUR.dot(np.concatenate((vxB.reshape(dim,3)[:,0],
                            vxB.reshape(dim,3)[:,1],vxB.reshape(dim,3)[:,2])))
    return cur_vxB

Parameters update Realized
Projectors Initialized
(3, 3)


In [52]:
curlB_f(B_phys)

array([[[[ 3.69778549e-32,  1.84889275e-32,  0.00000000e+00],
         [ 0.00000000e+00, -6.16297582e-33, -3.08148791e-33]],

        [[-2.46519033e-32, -1.23259516e-32, -3.85185989e-33],
         [-1.54074396e-32, -1.54074396e-33, -7.70371978e-34]],

        [[ 1.23259516e-32,  6.16297582e-33,  1.54074396e-33],
         [ 6.16297582e-33,  6.16297582e-33,  3.08148791e-33]],

        [[ 0.00000000e+00,  0.00000000e+00, -1.54074396e-33],
         [-6.16297582e-33, -3.08148791e-33, -1.54074396e-33]]],


       [[[ 1.23259516e-32,  6.16297582e-33, -7.70371978e-33],
         [-3.08148791e-32,  1.54074396e-32,  7.70371978e-33]],

        [[-1.23259516e-32, -6.16297582e-33, -2.29621275e-17],
         [-9.18485099e-17, -4.59242550e-17, -2.29621275e-17]],

        [[-9.18485099e-17, -4.59242550e-17, -2.29621275e-17],
         [-9.18485099e-17,  2.29621275e-17,  1.14810637e-17]],

        [[ 4.59242550e-17,  2.29621275e-17,  1.14810637e-17],
         [ 4.59242550e-17,  2.29621275e-17,  1.1481063

In [69]:
B_phys = lambda x,y,z: [np.sin(x*y),np.cos(y*z),0]
curlB_f(B_phys)

array([[[[-0.01086935,  0.00543468, -0.03687136],
         [ 0.14748543,  0.03002655, -0.01501327]],

        [[-0.04394111,  0.02197055, -0.00271734],
         [ 0.01086935, -0.07374271,  0.03687136]],

        [[ 0.06005309, -0.03002655, -0.01098528],
         [ 0.04394111, -0.00543468,  0.00271734]],

        [[-0.14748543,  0.07374271,  0.01501327],
         [-0.06005309, -0.02197055,  0.01098528]]],


       [[[-0.01086935,  0.00543468, -0.03687136],
         [ 0.14748543,  0.03002655, -0.01501327]],

        [[-0.04394111,  0.02197055,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]]],


       [[[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.  

In [62]:
curlb_ana=lambda x,y,z: [y*np.sin(y*z),0,-x*np.cos(x*y)]

In [79]:
x = [mapping.f(xi1, xi2, xi3, *args_map, j) for j in [1,2,3]]
x_t,y_t,z_t =np.meshgrid(x[0],x[1],x[2])
np.array(curlb_ana(x_t,y_t,z_t))

array([array([[[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.        , 0.12370198, 0.23971277],
        [0.        , 0.12370198, 0.23971277],
        [0.        , 0.12370198, 0.23971277],
        [0.        , 0.12370198, 0.23971277],
        [0.        , 0.12370198, 0.23971277]],

       [[0.        , 0.47942554, 0.84147098],
        [0.        , 0.47942554, 0.84147098],
        [0.        , 0.47942554, 0.84147098],
        [0.        , 0.47942554, 0.84147098],
        [0.        , 0.47942554, 0.84147098]],

       [[0.        , 1.02245814, 1.49624248],
        [0.        , 1.02245814, 1.49624248],
        [0.        , 1.02245814, 1.49624248],
        [0.        , 1.02245814, 1.49624248],
        [0.        , 1.02245814, 1.49624248]],

       [[0.        , 1.68294197, 1.81859485],
        [0.        